In [46]:
import string
import collections
import pandas as pd
import uuid
from sqlalchemy import create_engine
from datetime import datetime

In [12]:
quotes = [
    ("Oh! I want to put my arms around you, I ache to hold you close. Your ring is a great comfort. I look at it and think she does love me or I wouldn't be wearing it!", "Eleanor Roosevelt"),
    ("Understanding is a two-way street.", "Eleanor Roosevelt"),
    ("I think that somehow, we learn who we really are and then live with that decision.", "Eleanor Roosevelt"),
    ("I'd take the awe of understanding over the awe of ignorance any day.", "Douglas Adams"),
    ("Think for yourself and question authority.", "Timothy Leary"),
    ("We should be considerate to the living; to the dead we owe only the truth.", "Voltaire"),
]

In [59]:
def build_challenges_frequency(quotes, n):
    """
    Function to convert quotes to dictionaries with keys as replacements.
    
    Args:
    quotes: A list of tuples. Each tuple contains a quote and its author.
    n: The number of most frequent letters to be replaced.
    
    Returns:
    A list of dictionaries. Each dictionary contains the transformed quote, the replacement key and the author.
    """
    
    quote_dicts = []  # Initializing the output list
    
    for quote, author in quotes:  
        # Count the letter frequency
        letter_frequency = collections.Counter(quote.lower())
        
        # Sort by frequency, excluding non-letter characters
        sorted_letters = sorted([(letter, frequency) for letter, frequency in letter_frequency.items() 
                                 if letter in string.ascii_lowercase], key=lambda x: x[1], reverse=True)
        
        # Determine the n most frequent letters
        n_most_common = sorted_letters[:n]
        
        # Replace the letters with numbers from 1 to n in the order of their frequency
        key = {letter[0]: str(i+1) for i, letter in enumerate(n_most_common)}
        key_string = ''.join([letter for letter, _ in n_most_common])
        new_quote = "".join([key[letter] if letter in key else letter for letter in quote.lower()])
        
        # Create a dictionary and append it to the list
        quote_dict = {
            "challenge": new_quote,
            "text": quote,
            "key": key_string,
            "author": author,
            "type": "frequency",
        }
        quote_dicts.append(quote_dict)
    
    return quote_dicts


def build_challenges_names(quotes):
    """
    Function to convert quotes to dictionaries with keys as replacements.
    
    Args:
    quotes: A list of tuples. Each tuple contains a quote and its author.
    
    Returns:
    A list of dictionaries. Each dictionary contains the transformed quote, the replacement key (as a string) and the author.
    """
    
    quote_dicts = []  # Initializing the output list
    
    for quote, author in quotes:  
        # Get the first word of the author's name and convert it to lowercase
        first_name = author.split()[0].lower()
        
        # If the first name is shorter than 5 letters or contains duplicate letters, skip the quote
        if len(first_name) < 5 or len(set(first_name)) != len(first_name):
            continue
            
        # Replace the letters in the quote with numbers according to their position in the first name
        key_string = first_name
        key = {letter: str(i+1) for i, letter in enumerate(first_name)}
        new_quote = "".join([key[letter] if letter in key else letter for letter in quote.lower()])
        
        # Create a dictionary and append it to the list
        quote_dict = {
            "challenge": new_quote,
            "text": quote,
            "key": key_string,
            "author": author,
            "type": "name",
        }
        quote_dicts.append(quote_dict)
    
    return quote_dicts


def build_challenges_first_letters(quotes):
    """
    Function to convert quotes to dictionaries with keys as replacements.
    
    Args:
    quotes: A list of tuples. Each tuple contains a quote and its author.
    
    Returns:
    A list of dictionaries. Each dictionary contains the transformed quote, the replacement key (as a string) and the author.
    """
    
    quote_dicts = []  # Initializing the output list
    
    for quote, author in quotes:  
        # Get the first letter of each word in the quote, ensuring there are no repeats and length is not more than 9
        first_letters = [word[0].lower() for word in quote.split()]
        unique_letters = []
        for letter in first_letters:
            if letter not in unique_letters:
                unique_letters.append(letter)
            if len(unique_letters) == 9:
                break
                
        # Replace the letters in the quote with numbers according to their position in the unique_letters list
        key_string = ''.join(unique_letters)
        key = {letter: str(i+1) for i, letter in enumerate(unique_letters)}
        new_quote = "".join([key[letter.lower()] if letter.lower() in key else letter for letter in quote])
        
        # Create a dictionary and append it to the list
        quote_dict = {
            "challenge": new_quote,
            "text": quote,
            "key": key_string,
            "author": author,
            "type": "first",
        }
        quote_dicts.append(quote_dict)
    
    return quote_dicts


def build_challenges(quotes, n):
    """
    Wrapper function that uses both build_challenges_frequency and build_challenges_names.
    
    Args:
    quotes: A list of tuples. Each tuple contains a quote and its author.
    n: The number of most frequent letters to be replaced.
    
    Returns:
    A list of dictionaries. Each dictionary contains the transformed quote, the original quote, the replacement key and the author.
    """
    
    challenges = []  # Initializing the output list
    
    for quote, author in quotes:
        # Get the first word of the author's name and convert it to lowercase
        first_name = author.split()[0].lower()
        
        # If the first name is at least 5 letters and contains no duplicate letters, use build_challenges_names
        if len(first_name) >= 5 and len(set(first_name)) == len(first_name):
            challenges += build_challenges_names([(quote, author)])
            
        # If the first name is shorter than 5 letters or contains duplicate letters, use build_challenges_basic
        else:
            challenges += build_challenges_frequency([(quote, author)], n)
            
    return challenges


def insert_into_db(quote_dicts, db_config):
    """
    Function to insert quote dictionaries into a PostgreSQL database.
    
    Args:
    quote_dicts: A list of dictionaries. Each dictionary contains the transformed quote, the replacement key and the author.
    db_config: A dictionary containing the database configuration parameters (dbname, user, password, host).
    
    Returns:
    None.
    """
    
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(quote_dicts)
    
    # Add the required columns
    df['id'] = [str(uuid.uuid4()) for _ in range(len(df))]
    df['create_time'] = datetime.utcnow()
    df['update_time'] = datetime.utcnow()
    
    # Define the connection string
    conn_str = 'postgresql+psycopg2://{}:{}@{}/{}'.format(db_config['user'], 
                                                          db_config['password'], 
                                                          db_config['host'], 
                                                          db_config['dbname'])
    
    # Create the connection engine
    engine = create_engine(conn_str)
    
    # Insert the DataFrame into the database
    df.to_sql('challenges', engine, if_exists='replace', index=False, chunksize=1000)



In [60]:
challenges = build_challenges(quotes, 5)
print(f"created {len(challenges)} challenges")

created 6 challenges


In [62]:
db_config = {
    'dbname': 'test',
    'user': 'test',
    'password': 'pass',
    'host': 'localhost:6432',
}

insert_into_db(challenges, db_config)
print("data inserted")

data inserted
